In [1]:
import pandas as pd
import numpy as np
import sys
import pickle

In [2]:
from pitched_recommend import Recommender

In [13]:
playlists_data = pd.read_csv('/Users/meshchd/Downloads/recommender_browse_recommendations_playlists_data.csv')
playlists_list = pd.read_csv('/Users/meshchd/Downloads/recommender_browse_recommendations_playlists_list.csv')

In [14]:
playlists_list.head()

,territory,category_id,Position,playlist_uri,playlist_name
0,TR,pop,4,spotify:playlist:37i9dQZF1DWXoHqNlfcLJb,OLAY
1,TR,pop,6,spotify:playlist:37i9dQZF1DX9G2MXp8YYmm,Lokal
2,AT,pop,2,spotify:playlist:37i9dQZF1DX2YSAZIuAiB1,Swag!
3,CH,pop,14,spotify:playlist:37i9dQZF1DX2YSAZIuAiB1,Swag!
4,DE,pop,2,spotify:playlist:37i9dQZF1DX2YSAZIuAiB1,Swag!


In [15]:
playlists_data.head()

,playlist_name,listeners,country_owner,playlist_owner,ranking,pl_ranking,playlist_uri
0,Today's Top Hits,4110687,US,spotify,1,1,spotify:playlist:37i9dQZF1DXcBWIGoYBM5M
1,RapCaviar,2679420,US,spotify,1,1,spotify:playlist:37i9dQZF1DX0XUsuxWHRQd
2,Summer Hits,2149595,US,spotify,1,1,spotify:playlist:37i9dQZF1DX1gRalH1mWrP
3,Mega Hit Mix,1943838,US,spotify,1,1,spotify:playlist:37i9dQZF1DXbYM3nMM0oPk
4,Esquenta Sertanejo,1822133,BR,spotify,1,1,spotify:playlist:37i9dQZF1DXdSjVZQzv2tl


In [23]:
def get_browse_playlists(playlists_data, playlists_list):

    playlists_data['long_uri'] = ['spotify:user:'+str(x) for x in playlists_data.playlist_uri]

    data = playlists_list.merge(playlists_data, how='left', left_on='playlist_uri', right_on='long_uri')

### Get the 'same_as_country' column

    data['same_as_country'] = [1 if x==y else 0 for (x,y) in zip(data.territory,data.country_owner)]

### Get the 'count_appearance' column

    playlist_count = data.playlist_uri_x.value_counts()
    playlist_count = pd.DataFrame(playlist_count).reset_index().rename(columns={'index':'playlist_uri',
                                                                            'playlist_uri_x':'count_appearance'})
    data = data.merge(playlist_count, how='left', left_on = 'playlist_uri_x', right_on = 'playlist_uri')

### Get the 'local' column

    data['local'] = [1 if x<20 else 0 for x in data.count_appearance]

### Get the 'mean_listeners_cat_terr' column

    by_cat_terr = data.groupby(by=['territory','category_id']).listeners.mean()
    df_cat_terr = pd.DataFrame(by_cat_terr).reset_index()
    df_cat_terr = df_cat_terr.rename(columns = {'listeners':'mean_listeners_cat_terr'})

    data = data.merge(df_cat_terr, how='left', left_on = ['territory','category_id'], 
                      right_on = ['territory','category_id'])

### Get the 'popular' column

    data['popular'] = [1 if x>y else 0 for (x,y) in zip(data.listeners,data.mean_listeners_cat_terr)]

### Get the 'score' column with the overall score
    data['score'] = data.same_as_country + data.local + data.popular

    cols = ['territory', 'category_id', 'Position', 'playlist_uri',
       'playlist_name_x', 'listeners', 'country_owner', 'count_appearance',
        'mean_listeners_cat_terr', 'same_as_country', 'local', 'popular',
        'score']


    data = data[cols].sort_values(by=['territory','category_id','score'], ascending=False)
    data_final = pd.DataFrame(data.groupby(by=['territory','category_id']).head(10).reset_index())
    
    return data_final

In [24]:
playlists = get_browse_playlists(playlists_data,playlists_list)

In [25]:
playlists.head()

,index,territory,category_id,Position,playlist_uri,playlist_name_x,listeners,country_owner,count_appearance,mean_listeners_cat_terr,same_as_country,local,popular,score
0,63367,UY,workout,2,spotify:playlist:37i9dQZF1DX9ZKyQHcEFXZ,Para Entrenar,NaN,NaN,17,NaN,0,1,0,1
1,64260,UY,workout,1,spotify:playlist:37i9dQZF1DX4PvjnQT47nf,Ritmo de Selección,NaN,NaN,6,NaN,0,1,0,1
2,62323,UY,workout,11,spotify:playlist:37i9dQZF1DX4eRPd9frC1m,Hype,NaN,NaN,58,NaN,0,0,0,0
3,62407,UY,workout,13,spotify:playlist:37i9dQZF1DWSJHnPb1f0X3,Cardio,NaN,NaN,58,NaN,0,0,0,0
4,62445,UY,workout,7,spotify:playlist:37i9dQZF1DX70RN3TfWWJh,Workout,NaN,NaN,58,NaN,0,0,0,0


In [32]:
playlists.shape

(20643, 14)

In [4]:
with open('/Users/meshchd/Downloads/recommender_model_model_20190504.pkl', 'r+b') as pickle_file:
    model = pickle.load(pickle_file)

In [5]:
class MacOSFile(object):
    def __init__(self, f):
        self.f = f

    def __getattr__(self, item):
        return getattr(self.f, item)

    def read(self, n):
        if n >= (1 << 31):
            buffer = bytearray(n)
            pos = 0
            while pos < n:
                size = min(n - pos, 1 << 31 - 1)
                chunk = self.f.read(size)
                buffer[pos:pos + size] = chunk
                pos += size
            return buffer
        return self.f.read(n)

In [44]:
with open('/Users/meshchd/Downloads/recommender_model_model_20190504.pkl', "rb") as fin:
    model = pickle.load(MacOSFile(fin))

In [45]:
with open('/Users/meshchd/Downloads/recommender_metadata_metadata_dict.pkl', 'rb') as pickle_file:
    metadata = pickle.load(pickle_file)

In [46]:
all_territories = list(playlists.territory.unique())

In [47]:
rec = []

In [48]:
df=playlists.copy()

In [50]:
for territory in all_territories:
    #print("Producing recommendations for ", territory)
    terr_cat = list(df[df.territory==territory].category_id.unique())
    for category in terr_cat:
        #print("Producing recommendations for category ", category)
        terr_cat_playlists = list(df[(df.territory==territory)&(df.category_id==category)].playlist_uri.unique())
        rec.extend(model.rec_multiple_playlists(terr_cat_playlists, model.playlists_sparse, territory=territory,
                                          category = category, N=150))

In [51]:
rec

[(68606,
  1.3339078,
  'spotify:playlist:37i9dQZF1DX35oM5SPECmN',
  'UY',
  'workout'),
 (14004,
  1.2918768,
  'spotify:playlist:37i9dQZF1DWTl4y3vgJOXW',
  'UY',
  'workout'),
 (129396,
  1.2581462,
  'spotify:playlist:37i9dQZF1DWTl4y3vgJOXW',
  'UY',
  'workout'),
 (182, 1.2472253, 'spotify:playlist:37i9dQZF1DWTl4y3vgJOXW', 'UY', 'workout'),
 (12364,
  1.2163659,
  'spotify:playlist:37i9dQZF1DWTl4y3vgJOXW',
  'UY',
  'workout'),
 (30672,
  1.2145071,
  'spotify:playlist:37i9dQZF1DWTl4y3vgJOXW',
  'UY',
  'workout'),
 (163531,
  1.205905,
  'spotify:playlist:37i9dQZF1DWTl4y3vgJOXW',
  'UY',
  'workout'),
 (78904,
  1.2014798,
  'spotify:playlist:37i9dQZF1DWTl4y3vgJOXW',
  'UY',
  'workout'),
 (59563,
  1.1951997,
  'spotify:playlist:37i9dQZF1DX35oM5SPECmN',
  'UY',
  'workout'),
 (106091,
  1.191828,
  'spotify:playlist:37i9dQZF1DWTl4y3vgJOXW',
  'UY',
  'workout'),
 (68795, 1.190352, 'spotify:playlist:37i9dQZF1DWTl4y3vgJOXW', 'UY', 'workout'),
 (36533,
  1.1893079,
  'spotify:playli

In [30]:
for territory in all_territories:
    #print("Producing recommendations for ", territory)
    terr_cat = list(df[df.territory==territory].category_id.unique())
    for category in terr_cat:
        #print("Producing recommendations for category ", category)
        terr_cat_playlists = list(df[(df.territory==territory)&(df.category_id==category)].playlist_uri.unique())
        rec.extend(model.rec_multiple_playlists(terr_cat_playlists, model.playlists_sparse, territory=territory,
                                          category = category, N=150))

recommendations = [(model.isrcs[x[0]],float(x[1]),x[2],x[3],x[4]) for x in rec]
recs_final = [(x[0],x[1],metadata['track_artist'][x[0]], metadata['track_title'][x[0]], metadata['major_label'][x[0]], metadata['label_studio'][x[0]],
               metadata['original_release_date'][x[0]], metadata['genre_name'][x[0]], metadata['parent_genre_name'][x[0]], 
               x[2], x[3], x[4]) for x in list(recommendations)]

all_columns = ['isrc','score','artist','title','major_label','label_studio','original_release_date','genre_name',
               'parent_genre_name','seed_playlist_uri','territory','category']
rec_df = pd.DataFrame(recs_final, columns = all_columns)
rec_df_umg = rec_df[rec_df.major_label=='UMG']

final_columns = ['territory', 'category', 'score', 'isrc', 'seed_playlist_uri', 'artist', 'title', 'original_release_date']
rec_df_final = rec_df_umg[final_columns]

rec_df_final.to_csv('~/Downloads/manual_recommendations_20190408.csv', index=False, sep='\t')

In [52]:
recommendations = [(model.isrcs[x[0]],float(x[1]),x[2],x[3],x[4]) for x in rec]
recs_final = [(x[0],x[1],metadata['track_artist'][x[0]], metadata['track_title'][x[0]], metadata['major_label'][x[0]], metadata['label_studio'][x[0]],
               metadata['original_release_date'][x[0]], metadata['genre_name'][x[0]], metadata['parent_genre_name'][x[0]], 
               x[2], x[3], x[4]) for x in list(recommendations)]

all_columns = ['isrc','score','artist','title','major_label','label_studio','original_release_date','genre_name',
               'parent_genre_name','seed_playlist_uri','territory','category']
rec_df = pd.DataFrame(recs_final, columns = all_columns)
rec_df_umg = rec_df[rec_df.major_label=='UMG']

final_columns = ['territory', 'category', 'score', 'isrc', 'seed_playlist_uri', 'artist', 'title', 'original_release_date']
rec_df_final = rec_df_umg[final_columns]

rec_df_final.to_csv('~/Downloads/manual_recommendations_20190614.csv', index=False, sep='\t')

In [53]:
rec_df_final.head()

,territory,category,score,isrc,seed_playlist_uri,artist,title,original_release_date
0,UY,workout,1.333908,USC4R1503363,spotify:playlist:37i9dQZF1DX35oM5SPECmN,Nathaniel Rateliff & The Night Sweats,S.O.B.,2015-06-16
2,UY,workout,1.258146,QZ4JX1760724,spotify:playlist:37i9dQZF1DWTl4y3vgJOXW,Lil Baby,My Dawg,2017-07-18
7,UY,workout,1.201480,USUM71801625,spotify:playlist:37i9dQZF1DWTl4y3vgJOXW,2 Chainz,PROUD,2018-03-23
10,UY,workout,1.190352,QMJMT1300027,spotify:playlist:37i9dQZF1DWTl4y3vgJOXW,JAY Z,Tom Ford,2013-01-01
14,UY,workout,1.177753,QZ4JX1760824,spotify:playlist:37i9dQZF1DWTl4y3vgJOXW,Lil Baby,All Of A Sudden,2017-12-01


In [54]:
rec_df_final[rec_df_final.title=='Unknown']

,territory,category,score,isrc,seed_playlist_uri,artist,title,original_release_date


In [55]:
playlists[(playlists.territory=='DE')&(playlists.category_id=='rock')]

,index,territory,category_id,Position,playlist_uri,playlist_name_x,listeners,country_owner,count_appearance,mean_listeners_cat_terr,same_as_country,local,popular,score
15020,19924,DE,rock,1,spotify:playlist:37i9dQZF1DX4vth7idTQch,Rockhymnen,NaN,NaN,3,NaN,0,1,0,1
15021,20092,DE,rock,24,spotify:playlist:37i9dQZF1DX7fE85waV8CI,Klare Kante,NaN,NaN,3,NaN,0,1,0,1
15022,21094,DE,rock,11,spotify:playlist:37i9dQZF1DWZJhOVGWqUKF,Rock Rotation,NaN,NaN,3,NaN,0,1,0,1
15023,22593,DE,rock,37,spotify:playlist:37i9dQZF1DXbz5uvvU2C9n,Alternative Generation,NaN,NaN,4,NaN,0,1,0,1
15024,19247,DE,rock,35,spotify:playlist:37i9dQZF1DX1X7WV84927n,Hard Rock,NaN,NaN,113,NaN,0,0,0,0
15025,19264,DE,rock,2,spotify:playlist:37i9dQZF1DWWGFQLoP9qlv,Legendary,NaN,NaN,58,NaN,0,0,0,0
15026,19342,DE,rock,41,spotify:playlist:37i9dQZF1DWT2jS7NwYPVI,New Noise,NaN,NaN,58,NaN,0,0,0,0
15027,19443,DE,rock,10,spotify:playlist:37i9dQZF1DWWJOmJ7nRx0C,Rock Hard,NaN,NaN,72,NaN,0,0,0,0
15028,19520,DE,rock,9,spotify:playlist:37i9dQZF1DXcF6B6QPhFDv,Rock This,NaN,NaN,89,NaN,0,0,0,0
15029,19583,DE,rock,8,spotify:playlist:37i9dQZF1DX6xOPeSOGone,Soft Rock,NaN,NaN,58,NaN,0,0,0,0
